In [7]:
import caffe
import sys
sys.path.insert(0, '/home/gengshan/workJul/poseCaptioner_train/lib')
from dataLoader import fc7FrameSequenceGenerator
from hdf5_npstreamsequence_generator import HDF5SequenceWriter
from fileWriter import *
from utils import *

In [18]:
MAX_WORDS = 100
BUFFER_SIZE = 1 # TEXT streams
NUM_OUT_PER_CHUNK = 20
START_CHUNK = 0

SETTING = '/data/gengshan/pose_s2vt'
FRAMEFEAT_FILE_PATTERN = SETTING + '/splits/dataCsv_{0}.txt'
SENTS_FILE_PATTERN = SETTING + '/splits/dataTsv_{0}.txt'  # input paths
vocab_file = '/data/gengshan/pose_s2vt/whole_vocabulary.txt'                   #### change vocab file

LSTM_NET_FILE = 'protoFile/s2vt.words_to_preds.deploy.prototxt'                        #### change feature dim and 2 vocal dim in net file
RESULTS_DIR = SETTING + '/results'
NET_TAG = 's2vt_asl_pose_iter_8000'                                            #####change model
# NET_TAG = 's2vt_asl_pose_iter_40809'
# MODEL_FILE = SETTING + '/snapshots/stored/' + NET_TAG +'.caffemodel'
MODEL_FILE = SETTING + '/snapshots/' + NET_TAG +'.caffemodel'
STRATEGIES = [{'type': 'beam', 'beam_size': 1}]

data_split_name = 'train'                                                        #### change sv files

filenames =  [FRAMEFEAT_FILE_PATTERN.format(data_split_name),
               SENTS_FILE_PATTERN.format(data_split_name)]

In [19]:
fsg = fc7FrameSequenceGenerator(filenames, BUFFER_SIZE,
      vocab_file, max_words=MAX_WORDS, align=False, shuffle=False,
      pad=False, truncate=False, reverse = False)

Reading frame features from file: /data/gengshan/pose_s2vt/splits/dataCsv_train.txt
Reading sentences in: /data/gengshan/pose_s2vt/splits/dataTsv_train.txt
Initializing the vocabulary.
Initialized vocabulary from file with 13852 unique words (from 13851 total words in dataset).


In [20]:
video_gt_pairs = all_video_gt_pairs(fsg)  # target sentence
print 'Read %d videos pool feats' % len(fsg.vid_framefeats)
NUM_CHUNKS = 2  # (len(fsg.vid_framefeats)/NUM_OUT_PER_CHUNK) + 1
# add english inverted vocab 
vocab_list = ['<EOS>'] + fsg.vocabulary_inverted

Processed 0/37284 (0.000000%) lines
Processed 1/37284 (0.002682%) lines
Processed 100/37284 (0.268212%) lines
Processed 200/37284 (0.536423%) lines
Processed 300/37284 (0.804635%) lines
Processed 400/37284 (1.072846%) lines
Processed 500/37284 (1.341058%) lines
Processed 600/37284 (1.609269%) lines
Processed 700/37284 (1.877481%) lines
Processed 800/37284 (2.145693%) lines
Processed 900/37284 (2.413904%) lines
Found 1000 videos with 1000 captions
Read 37284 videos pool feats


In [21]:
caffe.set_mode_gpu()
caffe.set_device(0)
lstm_net = caffe.Net(LSTM_NET_FILE, MODEL_FILE, caffe.TEST)

In [22]:
for c in range(START_CHUNK, NUM_CHUNKS):
    chunk_start = c * NUM_OUT_PER_CHUNK
    chunk_end = (c + 1) * NUM_OUT_PER_CHUNK
    chunk = video_gt_pairs.keys()[chunk_start:chunk_end]
    html_out_filename = '%s/%s.%s.%d_to_%d.html' % \
      (RESULTS_DIR, data_split_name, NET_TAG, chunk_start, chunk_end)
    text_out_filename = '%s/%s.%s_' % \
      (RESULTS_DIR, data_split_name, NET_TAG)

    if not os.path.exists(RESULTS_DIR): os.makedirs(RESULTS_DIR)
    outputs = run_pred_iters(lstm_net, chunk, video_gt_pairs,
                fsg, strategies=STRATEGIES, display_vocab=vocab_list)

    html_out = to_html_output(outputs, vocab_list)
    html_out_file = open(html_out_filename, 'w')
    html_out_file.write(html_out)
    html_out_file.close() 
    # break
            

Num video features: 48 
Dimension of video features: (1, 20)
[[1, 1, 1, 1, 1, 0]]
[{'caption': [1, 1, 1, 1, 1, 0], 'gt': False, 'prob': [0.66988158, 0.26966834, 0.26535356, 0.35969102, 0.2960045, 0.35198367], 'source': {'beam_size': 1, 'type': 'beam'}}]
Generated caption (length 6, log_p = -6.321970, log_p_word = -1.053662):
<en_unk> <en_unk> <en_unk> <en_unk> <en_unk>.
Num video features: 45 
Dimension of video features: (1, 20)
[[1, 1, 1, 1, 1, 0]]
[{'caption': [1, 1, 1, 1, 1, 0], 'gt': False, 'prob': [0.66435832, 0.26776588, 0.2643593, 0.36212569, 0.29742193, 0.36735117], 'source': {'beam_size': 1, 'type': 'beam'}}]
Generated caption (length 6, log_p = -6.286826, log_p_word = -1.047804):
<en_unk> <en_unk> <en_unk> <en_unk> <en_unk>.
Num video features: 49 
Dimension of video features: (1, 20)
[[1, 1, 1, 1, 1, 0]]
[{'caption': [1, 1, 1, 1, 1, 0], 'gt': False, 'prob': [0.67165667, 0.27029401, 0.26572397, 0.35894775, 0.2956993, 0.34700367], 'source': {'beam_size': 1, 'type': 'beam'}}]


In [6]:
vocab_inds_to_sentence(vocab_list, outputs['--u_-w9WCG8_14'][0]['caption'])

KeyError: '--u_-w9WCG8_14'

In [39]:
for it, frame in enumerate(x):
    if it == 0:
        ifSeq = 0
    else:
        ifSeq = 1
    lstm_net.forward(frames_fc7=frame, cont_sentence=np.array([ifSeq]), input_sentence=np.array([0]), stage_indicator=np.array([0]))  # 0 is EOS, 1 is unique token

In [15]:
x = video_to_descriptor('--u_-w9WCG8_1', fsg)

In [80]:
lstm_net.blobs['cont_sentence'].data.shape

(100, 1)

In [18]:
x[1].shape

(1, 20)

In [25]:
video_gt_pairs['FvWTyzjD690_235'].shape

KeyError: 'FvWTyzjD690_235'

In [18]:
import h5py
data = h5py.File('/data/gengshan/pose_s2vt/hdf5/buffer_32_s2vt_100/test_batches/batch_0.h5', 'r')

In [34]:
list(data['cont_sentence'][:100, 0])

[0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [39]:
vocab_inds_to_sentence(vocab_list, [int(x) for x in data['target_sentence'][:100,0]])

'01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 have the same <en_unk> <EOS> 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01 01...'

In [88]:
probs = lstm_net.forward(frames_fc7=data['frame_fc7'][:100, :1, :], \
                 cont_sentence=data['cont_sentence'][:100, :1],\
                 input_sentence=data['input_sentence'][:100, :1],\
                 stage_indicator=data['stage_indicator'][:100, :1])['probs']